In [15]:
import os

import numpy as np

from keras import Sequential
from keras.utils import to_categorical
from keras.layers import LSTM, Dense, Dropout, Flatten, Activation

from pyknon.genmidi import Midi
from pyknon.music import NoteSeq, Note
from music21 import midi, stream, converter, note, chord, instrument

In [2]:
SEQUENCE_LENGTH = 5

In [17]:
melody = [10, 10, 10, 10, 11, 13]

def make_midi(notes, name, filepath):
    notes = [Note(note) for note in notes]
    midi = Midi(1, tempo=90)
    midi.seq_notes(notes, track=0)
    midi.write(filepath)
    
def play_midi(filepath):
    mf = midi.MidiFile()
    mf.open(filepath)
    mf.read()
    mf.close()
    stream = midi.translate.midiFileToStream(mf)
    stream.show('midi')

def load_midi(filepath):
    mf = midi.MidiFile()
    mf.open(filepath)
    mf.read()
    mf.close()
    return mf

In [4]:
play_midi('Omnibook/Midi/Anthropology.mid')

In [48]:
anthro = load_midi('Omnibook/Midi/Anthropology.mid')

In [3]:
def get_notes(songs):
    # from https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5
    notes = []
    for file in songs:
        midi = converter.parse(file)
        notes_to_parse = []
        try:
            parts = instrument.partitionByInstrument(midi)
        except Exception as e:
            print(e)
        if parts:
            notes_to_parse = parts.parts[0].recurse()
        else:
            notes_to_parse = midi.flat.notes
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
    return notes

In [4]:
songs = ['Omnibook/Midi/Anthropology.mid']
notes = get_notes(songs)
pitches = sorted(set(notes))
vocab_size = len(pitches)
pitch_indices = {pitch: idx for idx, pitch in enumerate(pitches)}
pitch_reverse_indices = {value: key for key, value in pitch_indices.items()}
pitch_sequence = [pitch_indices[note] for note in notes]

In [5]:
def make_training_sequences(notes, sequence_length, vocab_size):
    network_inputs = []
    network_outputs = []
    for i in range(len(notes) - sequence_length):
        input_sequence = notes[i:i+sequence_length]
        output_note = notes[i + sequence_length]
        network_inputs.append(input_sequence)
        network_outputs.append(output_note)
    inputs = np.reshape(network_inputs, (len(network_inputs), sequence_length, 1))
    inputs = inputs / vocab_size
    outputs = to_categorical(network_outputs)
    return inputs, outputs

In [6]:
inputs, outputs = make_training_sequences(pitch_sequence, SEQUENCE_LENGTH, vocab_size)

In [22]:
model = Sequential()  
model.add(LSTM(128, input_shape=inputs.shape[1:], return_sequences=True))  
model.add(Dropout(0.5))  
model.add(LSTM(128, return_sequences=True))  
model.add(Flatten())  
model.add(Dense(256))  
model.add(Dropout(0.5))  
model.add(Dense(vocab_size))  
model.add(Activation('softmax'))  
model.compile(loss='categorical_crossentropy', optimizer='adam')  

In [24]:
model.fit(inputs, outputs, epochs=100, batch_size=32)

Epoch 1/100
758/758 [==============================] - 1s 1ms/step - loss: 2.7903
Epoch 2/100
758/758 [==============================] - 1s 939us/step - loss: 2.7791
Epoch 3/100
758/758 [==============================] - 0s 652us/step - loss: 2.7688
Epoch 4/100
758/758 [==============================] - 0s 604us/step - loss: 2.7779
Epoch 5/100
758/758 [==============================] - 1s 769us/step - loss: 2.7581
Epoch 6/100
758/758 [==============================] - 1s 931us/step - loss: 2.7548
Epoch 7/100
758/758 [==============================] - 1s 821us/step - loss: 2.7374
Epoch 8/100
758/758 [==============================] - 1s 944us/step - loss: 2.7309
Epoch 9/100
758/758 [==============================] - 1s 1ms/step - loss: 2.7187
Epoch 10/100
758/758 [==============================] - 1s 770us/step - loss: 2.7196
Epoch 11/100
758/758 [==============================] - 1s 1ms/step - loss: 2.6815
Epoch 12/100
758/758 [==============================] - 1s 1ms/step - loss: 2.67

758/758 [==============================] - 1s 681us/step - loss: 1.2748
Epoch 98/100
758/758 [==============================] - 1s 691us/step - loss: 1.1814
Epoch 99/100
758/758 [==============================] - 0s 632us/step - loss: 1.1429
Epoch 100/100
758/758 [==============================] - 1s 681us/step - loss: 1.1986


In [10]:
def convert_to_notes(solo):
    offset = 0
    output_notes = []
    # create note and chord objects based on the values generated by the model
    for pattern in solo:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += 0.5
    return output_notes

In [11]:
def improvise(model, start_input, index_to_pitch, vocab_size, sequence_length, solo_length):
    solo = []
    solo.extend(start_input)
    
    for _ in range(solo_length):
        network_input = np.reshape(solo[-sequence_length:], (1, sequence_length, 1))
        network_input = network_input / vocab_size
        prediction = model.predict(network_input, verbose=False)
        prediction_note_index = np.argmax(prediction)
        solo.append(prediction_note_index)

    solo = [index_to_pitch[ind] for ind in solo]
    return solo  

In [13]:
def get_first_phrase(filename=None, sequence_length=SEQUENCE_LENGTH):
    songs_dir = 'Omnibook/Midi'
    if filename:
        song_filepath = os.path.join(songs_dir, filename)
        song_notes = get_notes([song_filepath])[:sequence_length]
        first_phrase = [pitch_indices[note] for note in song_notes]
        return filename, first_phrase
    
    song_filenames = os.listdir('Omnibook/Midi/')
    for filename in song_filenames:
        song_filepath = os.path.join(songs_dir, filename)
        song_notes = get_notes([song_filepath])[:sequence_length]
        try:
            first_phrase = [pitch_indices[note] for note in song_notes]
            return filename, first_phrase
        except Exception:
            pass

In [25]:
filename, start = get_first_phrase(filename='Yardbird_Suite.mid', sequence_length=SEQUENCE_LENGTH)
solo = improvise(
    model,
    start,
    index_to_pitch=pitch_reverse_indices,
    vocab_size=vocab_size,
    sequence_length=SEQUENCE_LENGTH,
    solo_length=50,
)
solo_notes = convert_to_notes(solo)
midi_stream = stream.Stream(solo_notes)
midi_stream.show('midi')


In [31]:
midi_stream.write('midi', 'bebop1.mid')

'bebop1.mid'

In [18]:
play_midi('Omnibook/Midi/Yardbird_Suite.mid')

In [34]:
pwd

'/Users/petervarshavsky/Documents/projects/bopnet'